# Paths

In [1]:
path = '/home/chayan/NeuroGDAVIS/'
res_plots = path + 'Results/Plots/'
path_emb = path + 'Data/Embeddings/'

# Importing libraries

In [2]:
import sys
sys.path.append(path)

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import keras
import ipynbname
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from utils import Datasets as ds
from keras.models import load_model
from utils import DownstreamAnalysis as DA

dataname = ipynbname.name().rsplit("_")[0]
txt = %pwd
methodname = txt.rsplit("/")[-1]

# Load labels and embeddings

In [3]:
y = ds.LoadData(dataname, DType='None', LType=True)
x_embedding = np.array(pd.read_csv(path_emb+dataname+'/'+dataname+'_'+methodname+'.csv', index_col=0, header=0))

# Downstream analysis

### k-NN

In [4]:
neighbour = [5, 15, 25, 35, 45]
Score_knn = []
for n in neighbour:
    Score_knn.append(DA.Knn(x_embedding, y, n))
Score_knn = pd.DataFrame(Score_knn, index=['neighbour '+ str(s) for s in neighbour],
                         columns = ['Accuracy', 'Precision','Recall','F1-Score'])
Score_knn

,Accuracy,Precision,Recall,F1-Score
neighbour 5,0.8115,0.811355,0.8115,0.809928
neighbour 15,0.8105,0.811637,0.8105,0.808686
neighbour 25,0.8055,0.806497,0.8055,0.803569
neighbour 35,0.8020,0.804090,0.8020,0.800197
neighbour 45,0.7995,0.802004,0.7995,0.797264


### Random Forest

In [5]:
n_tree = [20,40,60,80,100]
Score_rfc = []
for n in n_tree:
    Score_rfc.append(DA.RFC(x_embedding, y, n))
Score_rfc = pd.DataFrame(Score_rfc, index=['n_tree '+ str(s) for s in neighbour],
                         columns = ['Accuracy', 'Precision','Recall','F1-Score'])
Score_rfc

,Accuracy,Precision,Recall,F1-Score
n_tree 5,0.8005,0.800401,0.8005,0.799977
n_tree 15,0.8025,0.802902,0.8025,0.802301
n_tree 25,0.8105,0.810941,0.8105,0.810090
n_tree 35,0.8050,0.805322,0.8050,0.804656
n_tree 45,0.8060,0.806666,0.8060,0.805743


### k-Means

In [6]:
iteration = 10
Results_km = []

for i in tqdm(range(iteration), desc='Progress'):
    Results_km.append(DA.kmeans(x_embedding, y))
    
Results_km = pd.DataFrame(Results_km,columns = ['ARI', 'FMI','NMI'])
Results_km

Progress: 100%|█████████████████████████████████| 10/10 [00:00<00:00, 10.44it/s]


,ARI,FMI,NMI
0,0.272694,0.408027,0.355394
1,0.265787,0.406700,0.352823
2,0.271061,0.407750,0.353866
3,0.261778,0.402930,0.344059
4,0.235224,0.384309,0.319200
5,0.268547,0.405026,0.351505
6,0.231892,0.369226,0.316225
7,0.252324,0.395004,0.334475
8,0.271366,0.407252,0.354280
9,0.257895,0.400504,0.339561


### Agglomerative 

In [7]:
idx = ['ARI', 'FMI','NMI']
AglScore = pd.DataFrame(DA.Agglomerative(x_embedding, y), index = idx)    #seurat_clusters, Cluster, celltype.l2
AglScore.T

,ARI,FMI,NMI
0,0.232904,0.389984,0.319652
